In [ ]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import sparse
import json
import os
import ast
from mpl_toolkits.axes_grid1 import make_axes_locatable

### This notebook calculates the parameters for the San Francisco Bay
- homdeNodes: the dataset that indicates the home charger session in each trip chain
- Chain_df: trip chain
- mapping: mapping from nodes to counties

In [3]:
N_n = 9
N_t = 96

batteryCapacity = 56.5 # kWh

index_to_county = {
    0: 'San Francisco',
    1: 'San Mateo',
    2: 'Santa Clara',
    3: 'Alameda',
    4: 'Contra Costa',
    5: 'Solano',
    6: 'Napa',
    7: 'Sonoma',
    8: 'Marin'
}


# 从 JSON 文件中读取字典
with open('TripChains/California_Raw/chain_select_bay.json', 'r') as f:
    Chain_df = json.load(f)


q = np.load('TripChains/California_Raw/flows_select_bay.npy')
N_e = len(q)

s_init = np.zeros(N_e) + 0.5
P_max_home = 7 / batteryCapacity # AC charger: set as 7kW, convert to SoC: 7/56.5 
P_max_work = 15 / batteryCapacity # DC charger: set as 15kW, convert to SoC: 15/56.5

ratio_homeCharger_ori = pd.read_csv('geo_California/homeChargingRatio_bay.csv', index_col = 0)
ratio_homeCharger = {}
for county_ID, county_name in index_to_county.items():
    ratio_homeCharger[county_ID] = ratio_homeCharger_ori[ratio_homeCharger_ori['NAME'] == county_name]['Total Rate'].values[0]


In [4]:
flag = True
while flag == True:
    N_i = np.zeros(N_e)
    for e in range(N_e):
        N_i[e] = len(Chain_df[e])
    N_i = N_i.astype(int)
    error_chainInd = []
    for e in range(N_e):
        Chain_e = np.array(Chain_df[e])
        if Chain_e[-1, 3] < N_t - 1:
            start_charge, end_charge = np.zeros(N_i[e]), np.zeros(N_i[e])
            start_charge[:-1] = Chain_e[:-1, 3]
            end_charge[:-1] = Chain_e[1:, 2]
            start_charge[-1] = Chain_e[-1, 3]
            end_charge[-1] = N_t
            N_i_temp = N_i[e]
            location_charge = (Chain_e[:, 1]).astype(int)
        else:
            start_charge, end_charge = np.zeros(N_i[e]-1), np.zeros(N_i[e]-1)
            start_charge = Chain_e[:-1, 3]
            end_charge = Chain_e[1:, 2]
            N_i_temp = N_i[e] - 1
            location_charge = (Chain_e[:-1, 1]).astype(int)

        start_charge = start_charge.astype(int)
        end_charge = end_charge.astype(int)
        duration_charge = end_charge - start_charge
        if duration_charge.min() < 0:
            error_chainInd.append(e)
            Chain_df[e].pop()
    if len(error_chainInd) == 0:
        flag = False


N_i = np.zeros(N_e)
for e in range(N_e):
    N_i[e] = len(Chain_df[e])
N_i = N_i.astype(int)

In [5]:
E_init = 0.5
E_min = np.zeros(N_t)
E_min[-1] = 0.5
p_max_rec = np.zeros((2*N_e, N_t))
vio_rec = np.zeros(2*N_e)
q_rec = np.zeros(2*N_e)

Location_rec = np.zeros((2*N_e, N_t)) - 1
demand_cum_rec = np.zeros((2*N_e, N_t))

homeParkSession_duration = {}
nonHomeParkSession_duration = {}
nonHomeCounts = {}

L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1

home_node_update = {}
nonHomeParkDuration = {}
nonHomeParkStart = {}
nonHomeParkEnd = {}
error_chainInd = []
for e in range(N_e):
    Chain_e = np.array(Chain_df[e])
    if Chain_e[-1, 3] < N_t - 1:
        start_charge, end_charge = np.zeros(N_i[e]), np.zeros(N_i[e])
        start_charge[:-1] = Chain_e[:-1, 3]
        end_charge[:-1] = Chain_e[1:, 2]
        start_charge[-1] = Chain_e[-1, 3]
        end_charge[-1] = N_t
        N_i_temp = N_i[e]
        location_charge = (Chain_e[:, 1]).astype(int)
    else:
        start_charge, end_charge = np.zeros(N_i[e]-1), np.zeros(N_i[e]-1)
        start_charge = Chain_e[:-1, 3]
        end_charge = Chain_e[1:, 2]
        N_i_temp = N_i[e] - 1
        location_charge = (Chain_e[:-1, 1]).astype(int)

    start_charge = start_charge.astype(int)
    end_charge = end_charge.astype(int)
    duration_charge = end_charge - start_charge
    if duration_charge.min() < 0:
        error_chainInd.append(e)

    demand_e = np.zeros(N_t)
    for i in range(len(start_charge)):
        demand_e[start_charge[i]] += Chain_e[i, 4]

    demand_cum = np.cumsum(demand_e)
    demand_cum_rec[e] = demand_cum
    demand_cum_rec[N_e + e] = demand_cum

    nonHomeParkDuration[e] = []
    nonHomeParkStart[e] = []
    nonHomeParkEnd[e] = []
    nonHomeParkDuration[N_e + e] = []
    nonHomeParkStart[N_e + e] = []
    nonHomeParkEnd[N_e + e] = []

    if max(N_i_temp - 2, 0) not in nonHomeCounts.keys():
        nonHomeCounts[max(N_i_temp - 2, 0)] = q[e]
    else:
        nonHomeCounts[max(N_i_temp - 2, 0)] += q[e]

    for i in range(N_i_temp):
        Location_rec[e, start_charge[i]: end_charge[i]] = location_charge[i]
        Location_rec[N_e+e, start_charge[i]: end_charge[i]] = location_charge[i]
        
        if i == 0 or i == N_i_temp - 1:
            p_max_rec[e, start_charge[i]: end_charge[i]] = P_max_home
            if duration_charge[i] not in homeParkSession_duration.keys():
                homeParkSession_duration[duration_charge[i]] = q[e]
            else:
                homeParkSession_duration[duration_charge[i]] += q[e]
        else:
            nonHomeParkDuration[e].append(duration_charge[i])
            nonHomeParkStart[e].append(start_charge[i])
            nonHomeParkEnd[e].append(end_charge[i])
            nonHomeParkDuration[N_e + e].append(duration_charge[i])
            nonHomeParkStart[N_e + e].append(start_charge[i])
            nonHomeParkEnd[N_e + e].append(end_charge[i])
            if duration_charge[i] not in nonHomeParkSession_duration.keys():
                nonHomeParkSession_duration[duration_charge[i]] = q[e]
            else:
                nonHomeParkSession_duration[duration_charge[i]] += q[e]

    E_e_home = E_init - demand_cum + L @ p_max_rec[e] * 0.25
    E_e_pub = E_init - demand_cum
    vio_rec[e] = (E_e_home - E_min).min()
    vio_rec[N_e + e] = (E_e_pub - E_min).min()

    q_rec[e] = q[e] * ratio_homeCharger[location_charge[-1]]/100
    q_rec[N_e + e] = q[e] * (1 - ratio_homeCharger[location_charge[-1]]/100)


path_temp = 'Parameters/Bay/pubCharger'
isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
if not isExists:
    os.makedirs(path_temp)

q_rec_sparse = sparse.csr_matrix(q_rec)
demand_sparse = sparse.csr_matrix(demand_cum_rec)
Location_sparse = sparse.csr_matrix(Location_rec)

# sparse.save_npz(f"{path_temp}/q_rec.npz", q_rec_sparse)
# sparse.save_npz(f"{path_temp}/demand_cum.npz", demand_sparse)
# sparse.save_npz(f"{path_temp}/Location.npz", Location_sparse)



In [6]:
"""mirror"""
nonHomeParkSession_total = 0
for e in range(N_e, 2*N_e):
    nonHomeParkSession_total += len(nonHomeParkDuration[e]) * q_rec[e]

ratio_pubCharger_list = np.linspace(0.1, 1, 10)

nonHomeParkSession = 0
for ratio_pubCharger in ratio_pubCharger_list:
    ratio_current = 0
    for kk in range(100000):
        prority = np.argsort(vio_rec[N_e:])
        e = prority[0] + N_e
        demand_cum_temp = demand_cum_rec[e]
        q_actual = q_rec[e]
        try:
            i_temp = np.array(nonHomeParkDuration[e]).argmax()
            nonHomeParkDuration[e][i_temp] = -1
            p_max_rec[e, nonHomeParkStart[e][i_temp]: nonHomeParkEnd[e][i_temp]] = P_max_work
            p_max_rec[e-N_e, nonHomeParkStart[e-N_e][i_temp]: nonHomeParkEnd[e-N_e][i_temp]] = P_max_work # mirror
        except:
            q_actual = 0

        nonHomeParkSession += q_actual
        E_e = E_init - demand_cum_temp + L @ p_max_rec[e] * 0.25
        E_e_2 = E_init - demand_cum_temp + L @ p_max_rec[e-N_e] * 0.25 # mirror

        if len(nonHomeParkDuration[e]) == 0:
            vio_rec[e] = 100
            vio_rec[e-N_e] = 100
        elif np.array(nonHomeParkDuration[e]).mean() == -1:
                vio_rec[e] = 100
                vio_rec[e-N_e] = 100
        else:
            vio_rec[e] = (E_e - E_min).min()
            vio_rec[e-N_e] = (E_e_2 - E_min).min()
        ratio_current = nonHomeParkSession / nonHomeParkSession_total

        # if vio_rec.min() >= 0:
        #     print ('all chains become feasible', ratio_current)

        if ratio_current >= ratio_pubCharger:
            print (ratio_current, p_max_rec.sum(), vio_rec.min())
            break



    print(f'ratio_pubCharger: {ratio_pubCharger}, ratio_current: {ratio_current}')

    path_temp = 'Parameters/Bay/pubCharger'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
    if not isExists:
        os.makedirs(path_temp)

    """save the energy bounds"""
    p_max_sparse = sparse.csr_matrix(p_max_rec)
    # sparse.save_npz(f"{path_temp}/p_max_ratio={ratio_pubCharger:.1f}", p_max_sparse)


0.10001208766381413 111623.07964601772 -0.1622041728
ratio_pubCharger: 0.1, ratio_current: 0.10001208766381413
0.20007345459846565 143702.23008849556 -0.0697590528
ratio_pubCharger: 0.2, ratio_current: 0.20007345459846565
0.3000158762121417 168027.89380530975 -0.01903407359999998
ratio_pubCharger: 0.30000000000000004, ratio_current: 0.3000158762121417
0.4000166798179723 176218.76106194683 0.3580369232
ratio_pubCharger: 0.4, ratio_current: 0.4000166798179723
0.5000379135781908 182242.19469026552 0.4555424144
ratio_pubCharger: 0.5, ratio_current: 0.5000379135781908
0.6000005155967563 187642.47787610628 0.4844771936
ratio_pubCharger: 0.6, ratio_current: 0.6000005155967563
0.7000709501177531 192390.017699115 0.5
ratio_pubCharger: 0.7000000000000001, ratio_current: 0.7000709501177531
0.8000066508902657 196319.2212389381 0.5
ratio_pubCharger: 0.8, ratio_current: 0.8000066508902657
0.9000041757251208 200197.09734513276 0.5
ratio_pubCharger: 0.9, ratio_current: 0.9000041757251208
1.00000000000